# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {}
query["scores.Hygiene"] = 20

# Get the count of documents that match the query
count = 0
for doc in establishments.find(query):
    count += 1
print(count)

# Display the first document in the results using pprint
for doc in establishments.find(query):
    pprint(doc)
    break

In [ ]:
# Convert the result to a Pandas DataFrame
results = []
for doc in establishments.find(query):
    results.append(doc)
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
num_rows = len(df)
print((num_rows, len(df.columns)))

# Display the first 10 rows of the DataFrame
print(df.iloc[:10])

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {}
query["RatingValue"] = {"$gte": 4}
query["LocalAuthorityName"] = {"$regex": "London"}

# Manually count the number of documents that match the query
count = 0
for doc in establishments.find(query):
    count += 1
print(count)

# Display the first document in the results using a loop and pprint
for doc in establishments.find(query):
    pprint(doc)
    break

In [ ]:
# Convert the result to a Pandas DataFrame
results = []
for document in establishments.find(query):
    results.append(document)
df = pd.DataFrame(results)

# Manually calculate the number of rows and columns in the DataFrame
num_rows = len(df)
num_columns = len(df.columns)
print((num_rows, num_columns))

# Display the first 10 rows of the DataFrame
first_ten_rows = df.iloc[0:10]
print(first_ten_rows)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

# Build the query step by step
query = {}
query["RatingValue"] = 5
query["geocode.latitude"] = {"$gte": latitude - degree_search, "$lt": latitude + degree_search}
query["geocode.longitude"] = {"$gte": longitude - degree_search, "$lt": longitude + degree_search}

# Define sorting and limit
sort = [("scores.Hygiene", 1)]
limit = 5

# Execute the query and manually build the results list
results = []
for doc in establishments.find(query).sort(sort).limit(limit):
    results.append(doc)

# Print the results
for result in results:
    pprint(result)

In [ ]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(results)
df.head()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Build the match query step by step
match_query = {}
match_query['$match'] = {}
match_query['$match']["scores.Hygiene"] = 0

# Build the group query step by step
group_query = {}
group_query['$group'] = {}
group_query['$group']['_id'] = {"classification": "$LocalAuthorityName", "culture": "$culture"}
group_query['$group']['count'] = {}
group_query['$group']['count']['$sum'] = 1

# Build the sort query step by step
sort_values = {}
sort_values['$sort'] = {}
sort_values['$sort']['count'] = -1

# Assemble the pipeline manually
pipeline = []
pipeline.append(match_query)
pipeline.append(group_query)
pipeline.append(sort_values)

# Execute the pipeline and manually convert the results to a list
results_cursor = establishments.aggregate(pipeline)
results = []
for doc in results_cursor:
    results.append(doc)

# Print the number of documents in the result
num_results = len(results)
print(num_results)

# Print the first 10 results
for i in range(10):
    pprint(results[i])

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(df.shape)

# Display the first 10 rows of the DataFrame
df.head(10)